In [9]:
# # %%capture
# import sys
# import os
# if not os.path.exists('./city_temperature.csv'):
# #   Install Dependencies
#     !{sys.executable} -m pip install -r requirements.txt
#     !jupyter labextension install jupyterlab-plotly@4.14.3
#     !jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.14.3
#     !jupyter labextension update --all
#     !jupyter lab build
# #   Load dataset from kaggle
#     !kaggle datasets download -d sudalairajkumar/daily-temperature-of-major-cities
#     !unzip daily-temperature-of-major-cities.zip --force
#     if os.path.exists('./city_temperature.csv'):
#         !rm -rf daily-temperature-of-major-cities.zip

In [10]:
import geopandas as gp
import kaggle
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

init_notebook_mode(connected = True)

In [11]:
if os.path.exists('./city_temperature.csv'):
    df = pd.read_csv('./city_temperature.csv',low_memory=False)

In [12]:
df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [13]:
df['Date'] = pd.to_datetime(df.loc[:,'Month':'Year'],dayfirst=True,errors='coerce')

In [14]:
df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2,1995-01-01
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4,1995-01-02
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8,1995-01-03
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4,1995-01-04
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9,1995-01-05


In [15]:
world = gp.read_file(gp.datasets.get_path('naturalearth_lowres'))

In [16]:
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [17]:
gf = gp.GeoDataFrame(df.set_index('Country').join(world.set_index('name')),geometry='geometry')

In [18]:
gf.head()

,Region,State,City,Month,Day,Year,AvgTemperature,Date,pop_est,continent,iso_a3,gdp_md_est,geometry
Albania,Europe,NaN,Tirana,1,1,1995,-99.0,1995-01-01,3047987.0,Europe,ALB,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
Albania,Europe,NaN,Tirana,1,2,1995,-99.0,1995-01-02,3047987.0,Europe,ALB,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
Albania,Europe,NaN,Tirana,1,3,1995,-99.0,1995-01-03,3047987.0,Europe,ALB,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
Albania,Europe,NaN,Tirana,1,4,1995,-99.0,1995-01-04,3047987.0,Europe,ALB,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."
Albania,Europe,NaN,Tirana,1,5,1995,-99.0,1995-01-05,3047987.0,Europe,ALB,33900.0,"POLYGON ((21.02004 40.84273, 20.99999 40.58000..."


In [19]:
gf_95_1_1 = gf[gf.Date==dt.datetime(1995,1,1)]

@interact(d=widgets.SelectionSlider(
    options=gf['Date'].unique(),
    description='Date',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
))
def geo_plot(d):
    layout = dict(title=f'Global AvgTemperatures {pd.to_datetime(str(d)).strftime("%Y-%m-%d")}', geo=dict(showframe=False, projection={'type': 'natural earth'}))
    data = go.Choropleth(
        locations=gf[gf['Date']==d].index, 
        locationmode='country names', 
        z=gf[(gf['Date']==d)&(gf['AvgTemperature']!=-99)]['AvgTemperature'], 
        colorscale='matter', 
        colorbar={'title':'Avg. Temperature'},
        reversescale=True)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


interactive(children=(SelectionSlider(continuous_update=False, description='Date', options=(numpy.datetime64('…

In [20]:
@interact(name=widgets.Dropdown(
    options=df['Country'].unique(),
    description='Country',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
))
def plot_country(name):
    df_plot = df.drop(columns=['Year','Day','Month']).groupby(['Country',pd.Grouper(key='Date',freq='d')]).mean().reset_index()
    df_plot[(df_plot['Country']==name) & (df_plot['AvgTemperature']!=-99)].plot(x='Date',y='AvgTemperature',figsize=(30,10))
    plt.title(f'{name} Time Series Plot')
    plt.ylabel('AvgTemperature (c)')


interactive(children=(Dropdown(description='Country', options=('Algeria', 'Burundi', 'Benin', 'Central African…

intro/hypoth => 3
methods/cross-validation => 1
results/conclusion => 2
<!-- not-helping anyone =>  -->
regression => rando
k-fold => TA